In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
import os
os.chdir('D:/DataScienceWorkSpace/danish_sentence_similarty/src')

In [5]:
data_file = '../data/sentences.csv'

In [6]:
sentence_df = pd.read_csv(data_file)
sentence_df.head()

,id,text
0,10-12-176,"Vanddamp er en usynlig gas, der forekommer i s..."
1,10-13-182,Er der nogen herinde der har erfaring med at k...
2,10-14-29,Ved ikke lige hvordan disse er i størrelsen?
3,10-16-39,Dog kan jeg godt lide pang farver;)
4,10-17-297,Pengene bliver dog ofte først udbetalt efter 5...


In [7]:
target_sentence = sentence_df.loc[sentence_df['id']=='7-21-440','text']
target_sentence.values[0]

'Ifølge Dansk Kennelklub angriber muskelhunde dyr og mennesker cirka hver 14. dag.'

English translation for this sentence - __According to the Danish Kennel Club, muscular dogs attack animals and humans approximately every 14 days.__

## Option 1: Toenization > cleaning > Count > Similarity

In [126]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import DanishStemmer


from cleantext import clean

In [109]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(DanishStemmer().stem(item))
    return stems

def normalize_text(text):
    return clean(text,
     no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,      # replace all currency symbols with a special token
    no_punct=True,
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_currency_symbol="",
#     replace_with_punct=" ~PUNCT~ ",          # instead of removing punctuations you may replace them
#     replace_with_url=" ~URL~ ",
#     replace_with_email=" ~EMAIL~ ",
#     replace_with_phone_number=" ~PHONE~ ",
#     replace_with_number=" ~NUMBER~ ",
#     replace_with_currency_symbol=" ~CUR~ ",
    lang="de")

tokenize(normalize_text(target_sentence.values[0]))

['iflg',
 'dansk',
 'kennelklub',
 'angrib',
 'muskelhund',
 'dyr',
 'og',
 'mennesk',
 'cirka',
 'hver',
 'dag']

In [110]:
sentence_df_cp_op1 = sentence_df.copy()
sentence_df_cp_op1 = sentence_df_cp_op1.text.apply(normalize_text)
sentence_df_cp_op1

0       vanddamp er en usynlig gas der forekommer i st...
1       er der nogen herinde der har erfaring med at k...
2              ved ikke lige hvordan disse er i strrelsen
3                       dog kan jeg godt lide pang farver
4       pengene bliver dog ofte frst udbetalt efter da...
                              ...                        
4995    i dag i idrt skulle vi sa have bip test hvor j...
4996    p men tnkt nu hvis de bragte mere fra danskspr...
4997                             kathani hejsa allesammen
4998    weeeeeee jeg har mega optur pa de var begge mi...
4999    archon der blev jo ikke sagt noget om hvem der...
Name: text, Length: 5000, dtype: object

In [111]:
count_vect = CountVectorizer(tokenizer=tokenize, stop_words=stopwords.words('danish'))
count_vect_matrix = count_vect.fit_transform(sentence_df_cp_op1)
count_vect_matrix.shape

(5000, 13574)

In [112]:
cosine_matrix_cv = cosine_similarity(count_vect_matrix)

In [113]:
cosine_df_cv = pd.DataFrame(cosine_matrix_cv, columns = sentence_df['id'], index = sentence_df['id'])

In [114]:
target_cosine_array = cosine_df_cv.loc['7-21-440',:]
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
47-2-28       0.316228
51-33-2622    0.258199
36-0-1231     0.239046
42-26-238     0.239046
6-37-308      0.223607
45-86-368     0.223607
38-8-2425     0.223607
51-89-2877    0.223607
20-10-28      0.223607
Name: 7-21-440, dtype: float64

In [115]:
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [123]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'I dag'

In [122]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'Hver dag i december op til juleaften sendes et afsnit.'

In [118]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Det er der min arbejdsinteresse ligger, og det er noget, som jeg kan bruge hver dag.'

In [119]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Danskerne er blandt de europæere, som hver for sig producerer mest affald.'

In [120]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Hovedårsagen til, at Katrina blev så dyr, at skaderne blev så omfangsrige, er ganske enkelt, at der i dag bor langt flere mennesker i kystområderne end tidligere.'

In [127]:
tfidf_vect = TfidfVectorizer(tokenizer=tokenize, stop_words=stopwords.words('danish'))
tfidf_vect_matrix = tfidf_vect.fit_transform(sentence_df_cp_op1)
tfidf_vect_matrix.shape

D:\Anaconda\envs\danish_nlp\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['all', 'and', 'bliv', 'dis', 'eft', 'ell', 'hav', 'havd', 'hend', 'ikk', 'kun', 'mang', 'meg', 'nog', 'nogl', 'skul', 'und', 'vær'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5000, 13574)

In [128]:
cosine_matrix_ti = cosine_similarity(tfidf_vect_matrix)

In [129]:
cosine_df_ti = pd.DataFrame(cosine_matrix_ti, columns = sentence_df['id'], index = sentence_df['id'])

In [130]:
target_cosine_array = cosine_df_ti.loc['7-21-440',:]
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
47-2-28       0.216134
38-20-1059    0.180219
45-86-368     0.179803
36-0-1231     0.174177
38-78-1469    0.167071
34-2-723      0.166653
6-37-308      0.166108
40-81-2158    0.154190
51-33-2622    0.152718
Name: 7-21-440, dtype: float64

In [131]:
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [132]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'I dag'

In [133]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'Jeg ved, at niveauet er højt blandt alle angriberne i Serie A og kræver en toppræstation hver eneste gang.'

In [134]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Hvad er dyrets yndlingsfoder?'

In [135]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Det er der min arbejdsinteresse ligger, og det er noget, som jeg kan bruge hver dag.'

In [136]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Men det fik ikke den farlige angriber til at skåne City.'

## Option 2: BERT embedding for Danish > Similarity

In [8]:
from danish_bert_embeddings import DanishBertEmbeddings
embedder = DanishBertEmbeddings()

In [10]:
embedding = embedder.embed(target_sentence.values[0])

In [12]:
embedding.shape

torch.Size([768])

In [15]:
sentence_df_embed = sentence_df['text'].apply(embedder.embed)

In [27]:
sentence_embed_list = [t.numpy() for t in sentence_df_embed]

## Cosine similarity

In [28]:
cosine_matrix = cosine_similarity(sentence_embed_list)

In [34]:
cosine_df = pd.DataFrame(cosine_matrix, columns = sentence_df['id'], index = sentence_df['id'])

In [52]:
target_cosine_array = cosine_df.loc['7-21-440',:]
target_cosine_array.sort_values(ascending = False)[:10]

id
7-21-440      1.000000
38-21-615     0.713109
36-0-2615     0.705893
6-37-308      0.705858
51-3-3235     0.699858
36-7-877      0.694900
40-44-510     0.693219
3-28-67       0.692790
38-59-1673    0.692642
51-34-3098    0.692097
Name: 7-21-440, dtype: float32

In [53]:
top_10_similarity = target_cosine_array.sort_values(ascending = False)[:10].index.tolist()

In [61]:
sentence_df.loc[sentence_df.id == top_10_similarity[1],'text'].values[0]

'Han er fossilekspert, og foruden at være museumsinspektør ved Geomuseum Faxe er han forsker ved Københavns Universitet:" Over revet var der 200- 400 meter havvand, og Thoracosaurus har- akkurat som nulevende havkrokodiller- jagtet sit bytte i de øvre vandlag.'

In [62]:
sentence_df.loc[sentence_df.id == top_10_similarity[2],'text'].values[0]

'På 24 timer mellem 15. og 16. april 1949 fløj 1. 398 maskiner i alt 12. 849 tons fragt ind til den isolerede storby'

In [63]:
sentence_df.loc[sentence_df.id == top_10_similarity[3],'text'].values[0]

'Hovedårsagen til, at Katrina blev så dyr, at skaderne blev så omfangsrige, er ganske enkelt, at der i dag bor langt flere mennesker i kystområderne end tidligere.'

In [64]:
sentence_df.loc[sentence_df.id == top_10_similarity[4],'text'].values[0]

'Men indtil nu er alle teorierne kommet mere eller mindre til kort, da man hele tiden kan finde dyr, der har noget, der ligner menneskelige egenskaber.'

In [65]:
sentence_df.loc[sentence_df.id == top_10_similarity[5],'text'].values[0]

'Børshandlere er såvidt vides udstyret med samme biologiske profil som alle andre mennesker.'